In [ ]:
import cv2
import numpy as np
import pandas as pd
from random import randint
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from datetime import datetime
from keras.layers import Dense, Lambda, GlobalAveragePooling2D, Dropout, Activation, MaxPool2D
from keras.models import Model, Sequential, load_model, Input
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
#Read the csv file
df = pd.read_csv('data/labels.csv')

#Sort by frequency and extract top 10
dist = df.groupby('breed').count().rename(columns={'id':'freq'})
most_common = dist.sort_values(by='freq',ascending=False)
top_10 = [i for i in most_common[:10].index]

In [ ]:
dim = 150

In [ ]:
df = df[df['breed'].isin(top_10)]
X = df['id']
y = df['breed']
X_train_id, X_test_id, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_id.reset_index(drop=True, inplace=True)
X_test_id.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
train_len = len(y)
breeds = set(y)
class_len = len(breeds)
class_to_num = dict(zip(breeds, range(class_len)))
num_to_class = dict(zip(range(class_len), breeds))

In [ ]:
X_train_resized = np.zeros((len(X_train_id), dim, dim, 3),dtype=np.uint8)
y_train_categorical = np.zeros((len(X_train_id), 10), dtype=np.uint8)

In [ ]:
for i in tqdm(range(len(X_train_id))):
    image_path = 'data/train/{}.jpg'.format(X_train_id[i])
    img = Image.open(image_path)
    X_train_resized[i] = cv2.resize(cv2.imread(image_path), (dim,dim))
    y_train_categorical[i][class_to_num[y_train[i]]] = 1

In [ ]:
X_test_resized = np.zeros((len(X_test_id), dim, dim, 3),dtype=np.uint8)
y_test_categorical = np.zeros((len(X_test_id), 10),dtype=np.uint8)

In [ ]:
for i in tqdm(range(len(X_test_id))):
    image_path = 'data/train/{}.jpg'.format(X_test_id[i])
    img = Image.open(image_path)
    X_test_resized[i] = cv2.resize(cv2.imread(image_path), (dim,dim))
    y_test_categorical[i][class_to_num[y_test[i]]] = 1

In [ ]:
def feature_extraction(base_model, X):
    model = base_model(include_top=False, input_shape=(dim,dim,3), weights='imagenet')
    inputs = Input((dim, dim, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = model(x)
    x = GlobalAveragePooling2D()(x)
    model = Model(inputs,x)
    features = model.predict(X, batch_size=32, verbose=5)
    return features

In [ ]:
inception_features_train = feature_extraction(InceptionV3, X_train_resized)
np.save('data/features/10_inception_features_train.npy', inception_features_train)

In [ ]:
inception_features_test = feature_extraction(InceptionV3, X_test_resized)
np.save('data/features/10_inception_features_test.npy', inception_features_test)

In [ ]:
#Fully connected neural network
model = Sequential()
model.add(Dropout(0.3, input_shape=inception_features_train.shape[1:]))
model.add(Dense(10, activation='softmax'))

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
h = model.fit(inception_features_train, y_train_categorical, batch_size=128, epochs=50, validation_split=0.1)

In [ ]:
model.evaluate(inception_features_test, y_test_categorical, batch_size=128, verbose=1, sample_weight=None)

In [ ]:
print('minimum val_loss is {} at epoch {}'.format(min(h.history['val_loss']), np.argmin(h.history['val_loss'])+1))
print('maximum accuracy is {} at epoch {}'.format(max(h.history['val_acc']), np.argmax(h.history['val_acc'])+1))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')